In [41]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly
import plotly.express as px

In [34]:
df = pd.read_csv('iv_m_data_final.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [35]:
t_list = df.groupby('trade_date').count().index.tolist()

In [36]:
def data_pivot(df): # 数据透视
    df = df.reset_index()
    option_type = 'C' # 具有相同执行价格、相同剩余到期时间的看涨看跌期权隐含波动率相等，因此算一个就够了
    df = df[df['call_put']==option_type]
    df = df.drop(['ts_code','trade_date','c','s','r','call_put','q'],axis=1)
    df['t'] = df['t']*365
    df['t'] = df['t'].astype(int)
    df = df.pivot_table(index=["k"],columns=["t"],values=["iv"])
    df.columns = df.columns.droplevel(0)
    df.index.name = None
    df = df.reset_index()
    df = df.rename(columns={'index':'k'})

    return df

def fitting(df): # 多项式拟合    
    col_list = df.columns
    for i in range(df.shape[1]-1):
        x_col = col_list[0]
        y_col = col_list[i+1]
        df1 = df.dropna(subset=[y_col])
        
        x = df1.iloc[:,0]
        y = df1.iloc[:,i+1]

        degree = 2
                
        weights = np.polyfit(x, y, degree)
        model = np.poly1d(weights)
        predict = np.poly1d(model)
        x_given_list = df[pd.isnull(df[y_col]) == True][x_col].tolist()
        # 所有空值对应的k组成列表
        for x_given in x_given_list:
            y_predict = predict(x_given)
            df.loc[df[x_col]==x_given, y_col] = y_predict
    return df

In [37]:
def im_surface(df): # 波动率曲面作图
    # df = plot_df()
    df = fitting(df)    
    #df.to_excel('iv_fitting.xlsx')
    df = df.set_index('k')

    y = np.array(df.index)
    x = np.array(df.columns)
    fig = go.Figure(data=[go.Surface(z=df.values, x=x, y=y)])

    fig.update_layout(scene = dict(
                    xaxis_title='剩余期限',
                    yaxis_title='执行价格',
                    zaxis_title='隐含波动率'),
                    width=1400,
                    margin=dict(r=20, b=10, l=10, t=10),
                    )
    # fig.write_image("fig1.jpg")
    plotly.offline.plot(fig)

def smile_plot(df): # 波动率微笑作图
    # df = plot_df()
    df = df.set_index('k')
    df = df.stack().reset_index()
    df.columns = ['k', 'days', 'iv']
    fig = px.line(df, x="k", y="iv", color="days",line_shape="spline")
    plotly.offline.plot(fig)

取某一天

In [46]:
t1 = t_list[5]

In [63]:
t1 = '2018-08-10'

In [64]:
d_test = data_pivot(df[df.trade_date==t1])
fitting(d_test)

t,k,11,47,138,229
0,2.20,0.293631,0.275803,0.251219,0.232909
1,2.25,0.283970,0.263496,0.246170,0.231533
2,2.30,0.275498,0.257114,0.243721,0.229027
3,2.35,0.266205,0.253063,0.236790,0.228982
4,2.40,0.262436,0.247173,0.236080,0.225607
5,2.45,0.258507,0.243885,0.233591,0.223478
6,2.50,0.255844,0.240093,0.229095,0.223314
7,2.55,0.251282,0.238607,0.226992,0.223385
8,2.60,0.250053,0.235634,0.226749,0.221583
9,2.65,0.249344,0.235420,0.222383,0.217230


In [65]:
im_surface(d_test)